##### Copyright 2018 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow를 사용하여 분산 훈련하기

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/distributed_training"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>   </td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/distributed_training.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/distributed_training.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/distributed_training.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

## Overview

`tf.distribute.Strategy`는 여러 GPU, 여러 머신 또는 TPU에 훈련을 분산하는 TensorFlow API입니다. 이 API를 사용하면 최소한의 코드 변경으로 기존 모델 및 훈련 코드를 분산 처리할 수 있습니다.

`tf.distribute.Strategy`는 다음과 같은 주요 목표를 염두에 두고 설계되었습니다.

- 사용하기 쉬우며 연구원, 머신러닝 엔지니어 등 여러 사용자 세그먼크 지원.
- 뛰어난 성능을 즉시 제공.
- 전략 간 전환이 용이.

Keras `Model.fit`과 같은 고수준 API를 사용하는 `tf.distribute.Strategy`뿐만 아니라 [사용자 정의 훈련 루프](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch)도 사용하여 훈련을 분산할 수 있습니다(일반적으로 TensorFlow를 사용하는 모든 계산도 포함).

TensorFlow 2.x에서는 프로그램을 긴급하게 실행하거나 [`tf.function`](function.ipynb)을 사용하여 그래프에서 실행할 수 있습니다. `tf.distribute.Strategy`는 이러한 실행 모드를 모두 지원하지만 `tf.function`에서 가장 잘 작동합니다. Eager 모드는 디버깅 목적으로만 권장되며 `tf.distribute.TPUStrategy`에는 지원되지 않습니다. 이 가이드는 훈련에 초점을 맞추고 있지만 이 API는 다른 플랫폼에서 평가 및 예측을 분산하는 데에도 사용할 수 있습니다.

TensorFlow의 기본 구성요소가 전략을 인식하도록 변경되었기 때문에 코드를 약간만 변경하면 `tf.distribute.Strategy`를 사용할 수 있습니다. 이는 변수, 레이어, 모델, 옵티마이저, 메트릭, 요약 및 체크포인트를 포함합니다.

이 가이드에서는 다양한 유형의 전략과 다양한 상황에서 전략을 사용하는 방법에 대해 배우게 됩니다. 성능 문제를 디버그하는 방법을 알아보려면 [ TensorFlow GPU 성능 최적화](gpu_performance_analysis.md) 가이드를 확인하세요.

참고: 이 개념을 보다 깊이 이해하려면 심층 프레젠테이션([Inside TensorFlow: `tf.distribute.Strategy`](https://youtu.be/jKV53r9-H14))을 시청하세요. 이 동영상은 특히 자신만의 훈련 루프를 작성할 계획을 가진 분들께 권장합니다.


## TensorFlow 설정하기

In [ ]:
import tensorflow as tf

## 전략 유형

`tf.distribute.Strategy`는 다양한 축을 따라 여러 가지 사용 사례를 다룹니다. 일부 조합은 현재 지원되며 다른 조합은 향후에 추가될 예정입니다. 일부 축에 대한 내용은 다음과 같습니다.

- *동기식 및 비동기식 훈련:* 데이터 병렬 처리로 훈련을 분산하는 두 가지 일반적인 방법입니다. 동기화 훈련에서 모든 작업자는 동기화된 입력 데이터의 다른 슬라이스에 대해 훈련하고 각 단계에서 그래디언트를 집계합니다. 비동기 훈련에서 모든 작업자는 입력 데이터에 대해 독립적으로 훈련하고 변수를 비동기적으로 업데이트합니다. 일반적으로 동기화 훈련은 매개변수 서버 아키텍처를 통해 all-reduce 및 async로 지원됩니다.
- *하드웨어 플랫폼:* 하나의 머신에서 여러 GPU로, 네트워크에서 여러 머신으로(각각 0개 이상의 GPU 포함) 또는 Cloud TPU로 훈련을 확장할 수 있습니다.

이러한 사용 사례를 지원하기 위해 TensorFlow에서 사용할 수 있는 `MirroredStrategy`, `TPUStrategy`, `MultiWorkerMirroredStrategy`, `ParameterServerStrategy`, `CentralStorageStrategy` 및 기타 전략이 있습니다. 다음 섹션에서는 TensorFlow의 각 시나리오에서 어떠한 항목이 지원되는지 설명합니다. 다음은 간략한 개요입니다.

훈련 API | `MirroredStrategy` | `TPUStrategy` | `MultiWorkerMirroredStrategy` | `CentralStorageStrategy` | `ParameterServerStrategy`
:-- | :-- | :-- | :-- | :-- | :--
**Keras `Model.fit`** | 지원 | 지원 | 지원 | 실험 기능으로 지원 | 실험 기능으로 지원
**사용자 정의 훈련 루프** | 지원 | 지원 | 지원 | 실험 기능으로 지원 | 실험 기능으로 지원
**Estimator API** | 제한적인 지원 | 지원 안 함 | 제한적인 지원 | 제한적인 지원 | 제한적인 지원

참고: [실험적 지원](https://www.tensorflow.org/guide/versions#what_is_not_covered)은 API에 호환성 보장이 적용되지 않음을 의미합니다.

경고: Estimator 지원은 제한적입니다. 기본 훈련 및 평가는 실험적이며 스캐폴드와 같은 고급 기능은 구현되지 않습니다. 사용 사례가 적용되지 않는 경우 Keras 또는 사용자 정의 훈련 루프를 사용해야 합니다. 새 코드에는 Estimator를 사용하지 않는 것이 좋습니다. Estimator는 올바르게 작성하기가 더 어려운 `v1.Session` 스타일 코드를 실행하며 특히 TF 2 코드와 결합할 때 예기치 않게 작동할 수 있습니다. Estimator는 당사의 [호환성 보장](https://tensorflow.org/guide/versions) 대상에 해당하지만 보안 취약점 외에는 수정 사항이 제공되지 않습니다. 자세한 내용은 [이전 가이드](https://tensorflow.org/guide/migrate)를 참조하세요.

### MirroredStrategy

`tf.distribute.MirroredStrategy`는 하나의 머신으로 여러 GPU로 수행하는 동기식 분산 훈련을 지원합니다. GPU 장치당 하나의 복제본을 생성합니다. 모델의 각 변수는 모든 복제본에서 미러링됩니다. 변수들은 다 함께 `MirroredVariable`이라는 단일 개념적 변수를 형성합니다. 모든 변수에 동일한 업데이트를 적용하여 서로 동기화된 상태로 유지됩니다.

여러 장치에 변수의 변경사항을 전달하기 위하여 효율적인 올 리듀스(all-reduce) 알고리즘을 사용합니다. 올 리듀스 알고리즘은 모든 장치에 걸쳐 텐서를 모은 다음, 그 합을 구하여 다시 각 장비에 제공합니다. 이 통합된 알고리즘은 매우 효율적이어서 동기화의 부담을 많이 덜어낼 수 있습니다. 장치 간에 사용 가능한 통신 방법에 따라 다양한 올 리듀스 알고리즘과 구현이 있습니다. 기본적으로 NVIDIA Collective Communication Library ([NCCL](https://developer.nvidia.com/nccl))를 올 리듀스 구현으로 사용합니다. 몇 가지 다른 옵션 중에서 선택하거나 직접 작성할 수 있습니다.

`MirroredStrategy`를 생성하는 가장 간단한 방법은 다음과 같습니다.

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()

이렇게 하면 TensorFlow에 표시되는 모든 GPU를 사용하고 NCCL을 장치 간 통신 수단으로 사용하는 `MirroredStrategy` 인스턴스가 생성됩니다.

머신에서 GPU 중 일부만 사용하려면 다음과 같이 하면 됩니다.

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

장치 간 통신을 재정의하려면 `tf.distribute.CrossDeviceOps`의 인스턴스를 제공하여 `cross_device_ops` 인수를 사용하면 됩니다. 현재 `tf.distribute.HierarchicalCopyAllReduce` 및 `tf.distribute.ReductionToOneDevice`는 기본값인 `tf.distribute.NcclAllReduce` 이외의 두 가지 옵션입니다.

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy(
    cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())

### TPUStrategy

`tf.distribute.TPUStrategy`를 사용하면 [Tensor Processing Units(TPU)](tpu.ipynb)에서 TensorFlow 훈련을 실행할 수 있습니다. TPU는 머신 훈련 워크로드를 극적으로 가속화하도록 설계된 Google의 특수 ASIC입니다. TPU는 [Google Colab](https://colab.research.google.com/), [TPU Research Cloud](https://sites.research.google/trc/), [Cloud TPU](https://cloud.google.com/tpu)에서 사용할 수 있습니다.

`tf.distribute.experimental.MultiWorkerMirroredStrategy`은 `MirroredStrategy`와 매우 비슷합니다. 다중 워커를 이용하여 동기 분산 훈련을 합니다. 각 워커는 여러 개의 GPU를 사용할 수 있습니다. `MirroredStrategy`처럼 모델에 있는 모든 변수의 복사본을 모든 워커의 각 장치에 만듭니다.

`TPUStrategy`를 인스턴스화하는 방법은 다음과 같습니다.

참고: Colab에서 TPU 코드를 실행하려면 TPU를 Colab 런타임으로 선택해야 합니다. 전체 예시는 [TPU 사용하기](tpu.ipynb) 가이드를 참조하세요.

```python
cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
    tpu=tpu_address)
tf.config.experimental_connect_to_cluster(cluster_resolver)
tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
tpu_strategy = tf.distribute.TPUStrategy(cluster_resolver)
```

Note: 이 전략은 아직 개선 중이고 더 많은 경우에 쓸 수 있도록 만들고 있기 때문에, <a><code>실험 기능</code></a>으로 지원됩니다. 따라서 다음에 API가 바뀔 수 있음에 유념하십시오.

Cloud TPU에서 사용하려는 경우:

- `tpu` 인수에 TPU 리소스의 이름을 지정해야 합니다.
- 프로그램을 *시작*할 때 명시적으로 TPU 시스템을 초기화해야 합니다. 이 작업은 TPU를 계산에 사용하기 전에 필요합니다. TPU 시스템을 초기화하면 TPU 메모리도 지워지므로 상태 손실을 방지하기 위해 이 단계를 먼저 완료하는 것이 중요합니다.

### MultiWorkerMirroredStrategy

`tf.distribute.MultiWorkerMirroredStrategy`는 `MirroredStrategy`와 매우 유사합니다. 이는 잠재적으로 여러 GPU가 있는 여러 작업자에 걸쳐 동기식 분산 훈련을 구현합니다. `tf.distribute.MirroredStrategy`와 유사하게 모든 작업자에 걸쳐 각 장치의 모델에 있는 모든 변수의 복사본을 만듭니다.

다중 워커(multi-worker)들 사이에서는 올 리듀스(all-reduce) 통신 방법으로 [CollectiveOps](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/ops/collective_ops.py)를 사용하여 변수들을 같은 값으로 유지합니다. 수집 연산(collective op)은 텐서플로 그래프에 속하는 연산 중 하나입니다. 이 연산은 하드웨어나 네트워크 구성, 텐서 크기에 따라 텐서플로 런타임이 지원하는 올 리듀스 알고리즘을 자동으로 선택합니다.

In [ ]:
strategy = tf.distribute.MultiWorkerMirroredStrategy()

`MultiWorkerMirroredStrategy`에는 장치 간 사용할 수 있는 수집 연산 구현이 두 가지 있습니다. `CommunicationImplementation.RING`는 [RPC](https://en.wikipedia.org/wiki/Remote_procedure_call)를 기반으로 하며 CPU와 GPU 모두를 지원합니다.<br>gRPC를 사용한 링 네트워크 기반의 수집 연산입니다. `CommunicationImplementation.NCCL`은 NCCL을 사용하고 GPU에서 최신의 성능을 제공하지만 CPU는 지원하지 않습니다. `CollectiveCommunication.AUTO`는 선택을 Tensorflow로 연기합니다. 지정하는 방법은 다음과 같습니다.


In [ ]:
communication_options = tf.distribute.experimental.CommunicationOptions(
    implementation=tf.distribute.experimental.CommunicationImplementation.NCCL)
strategy = tf.distribute.MultiWorkerMirroredStrategy(
    communication_options=communication_options)

멀티 GPU 교육 대비 다중 작업자 훈련을 진행하는 주요 차이점 중 하나는 다중 작업자 설정입니다. TF_CONFIG 환경 변수는 클러스터의 일부인 각 작업자에 클러스터 구성을 지정하는 TensorFlow의 표준 방법입니다. 이 문서의 [TF_CONFIG 설정하기 섹션](#TF_CONFIG)에서 자세히 알아보세요.

`MultiWorkerMirroredStrategy`에 대한 자세한 내용은 다음 튜토리얼을 참조하세요.

- [Keras Model.fit를 사용한 다중 작업자 훈련](../tutorials/distribute/multi_worker_with_keras.ipynb)
- [사용자 정의 훈련 루프를 사용한 다중 작업자 훈련](../tutorials/distribute/multi_worker_with_ctl.ipynb)

### ParameterServerStrategy

매개변수 서버 훈련은 여러 머신에서 모델 훈련을 확장하는 일반적인 데이터 병렬 방법입니다. 매개변수 서버 훈련 클러스터는 작업자와 매개변수 서버로 구성되어 있습니다. 변수는 매개변수 서버에서 생성되며 각 단계에서 작업자가 읽고 업데이트합니다. 자세한 내용은 [매개변수 서버 훈련](../tutorials/distribute/parameter_server_training.ipynb) 가이드를 확인하세요.

TensorFlow 2에서 매개변수 서버 훈련은 `tf.distribute.experimental.coordinator.ClusterCoordinator` 클래스를 통해 센트럴 코디네이터 기반 아키텍처를 사용합니다.

이 구현에서 `worker` 및 `parameter server` 작업은 코디네이터의 작업을 수신하는 `tf.distribute.Server`를 실행합니다. 코디네이터는 리소스를 생성하고, 훈련 작업을 발송하고, 체크포인트를 작성하고, 작업 실패를 처리합니다.

코디네이터에서 실행되는 프로그래밍에서 `ParameterServerStrategy` 개체를 사용하여 훈련 단계를 정의하고 `ClusterCoordinator`를 사용하여 원격 작업자에게 훈련 단계를 전달합니다. 이를 생성하는 가장 간단한 단계는 다음과 같습니다.

```python
strategy = tf.distribute.experimental.ParameterServerStrategy(
    tf.distribute.cluster_resolver.TFConfigClusterResolver(),
    variable_partitioner=variable_partitioner)
coordinator = tf.distribute.experimental.coordinator.ClusterCoordinator(
    strategy)
```

`ParameterServerStrategy`에 대해 자세히 알아보려면 [Keras Model.fit 및 사용자 정의 훈련 루프를 사용하는 매개변수 서버 훈련](../tutorials/distribute/parameter_server_training.ipynb) 가이드를 확인하세요.

참고: `TFConfigClusterResolver`를 사용하는 경우 `'TF_CONFIG'` 환경 변수를 구성해야 합니다. 이는 `MultiWorkerMirroredStrategy`의 <a href="#TF_CONFIG" data-md-type="link">`'TF_CONFIG'`</a>와 유사하지만 추가 주의 사항이 있습니다.

TensorFlow 1에서 `ParameterServerStrategy`는 `tf.compat.v1.distribute.experimental.ParameterServerStrategy` 기호를 통해 Estimator에서만 사용할 수 있습니다.

참고: 이 전략은 현재 개발 중이므로 [`experimental`](https://www.tensorflow.org/guide/versions#what_is_not_covered)입니다.

### CentralStorageStrategy

`tf.distribute.experimental.CentralStorageStrategy`는 동기식 훈련도 수행합니다. 변수는 미러링되지 않고 대신 CPU에 배치되며 작업이 모든 로컬 GPU에 복제됩니다. GPU가 하나만 있는 경우 모든 변수와 연산은 해당 GPU에 배치됩니다.

다음을 수행하여 `CentralStorageStrategy`의 인스턴스를 생성합니다.


In [ ]:
central_storage_strategy = tf.distribute.experimental.CentralStorageStrategy()

이렇게 하면 보이는 모든 GPU와 CPU를 사용하는 `CentralStorageStrategy` 인스턴스가 생성됩니다. 복제본의 변수에 대한 업데이트는 변수에 적용되기 전에 집계됩니다.

참고: 이 전략은 현재 작업을 진행하고 있기에 [`experimental`](https://www.tensorflow.org/guide/versions#what_is_not_covered)입니다.

### 기타 전략

위의 전략 외에도 `tf.distribute` API를 사용할 때 프로토타이핑 및 디버깅에 유용할 수 있는 두 가지 기타 전략이 있습니다.

#### 기본 전략

기본 전략은 명시적인 분산 전략이 범위에 없을 때 존재하는 분산 전략입니다. `tf.distribute.Strategy` 인터페이스를 구현하지만 그냥 통과하며 실제 분산 기능은 제공하지 않습니다. 예를 들어 `Strategy.run(fn)`은 단순히 `fn`을 호출합니다. 이 전략을 사용하여 작성한 코드는 전략 없이 작성한 코드와 똑같이 작동해야 합니다. 이를 "no-op" 전략이라 생각할 수 있습니다.

기본 전략은 싱글톤이며 더 이상 인스턴스를 만들 수 없습니다. 명시적 전략 범위 외부에서 `tf.distribute.get_strategy`를 사용하여 가져올 수 있습니다(명시적 전략 범위 내에서 현재 전략을 가져올 때 사용할 수 있는 API와 동일).

In [ ]:
default_strategy = tf.distribute.get_strategy()

이 전략은 다음과 같은 두 가지 주요 목적을 수행합니다.

- 분산 인식 라이브러리 코드를 무조건으로 작성할 수 있습니다. 예를 들어 `tf.optimizer`에서 `tf.distribute.get_strategy`를 사용하고 이 전략을 사용하여 그래디언트를 줄일 수 있습니다. 그러면 항상 `Strategy.reduce` API를 호출할 수 있는 전략 개체를 반환합니다.


In [ ]:
# In optimizer or other library code
# Get currently active strategy
strategy = tf.distribute.get_strategy()
strategy.reduce("SUM", 1., axis=None)  # reduce some values

- 라이브러리 코드와 유사하게 조건부 논리 없이 분산 전략을 사용하거나 사용하지 않고 최종 사용자의 프로그램을 작성하는 데 기본 전략을 사용할 수 있습니다. 다음은 이를 보여주는 예제 코드 조각입니다.

In [ ]:
if tf.config.list_physical_devices('GPU'):
  strategy = tf.distribute.MirroredStrategy()
else:  # Use the Default Strategy
  strategy = tf.distribute.get_strategy()

with strategy.scope():
  # Do something interesting
  print(tf.Variable(1.))

#### OneDeviceStrategy

`tf.distribute.OneDeviceStrategy`는 모든 변수와 계산을 지정한 단일 장치에 배치하는 전략입니다.

```python
strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
```

이 전략은 여러 면에서 기본 전략과 다릅니다. 기본 전략에서 변수 배치 논리는 분산 전략 없이 TensorFlow를 실행하는 것과 비교할 경우 변경되지 않은 상태로 유지됩니다. 그러나 `OneDeviceStrategy`를 사용하면 해당 범위에서 생성된 모든 변수가 지정한 장치에 명시적으로 배치됩니다. 또한 `OneDeviceStrategy.run`을 통해 호출한 모든 함수도 지정한 장치에 배치됩니다.

이 전략을 통해 분산된 입력은 지정한 장치로 미리 가져옵니다. 기본 전략에는 입력 분산이 없습니다.

기본 전략과 유사하게 이 전략은 실제로 여러 장치/머신에 분산하는 다른 전략으로 전환하기 전에 코드를 테스트하는 데 사용할 수도 있습니다. 이렇게 하면 다소 기본 전략 이상의 분산 전략 머신만 실행하지만 예컨대 `MirroredStrategy` 또는 `TPUStrategy`등은 완전히 사용하지는 않습니다. 전략이 없는 것처럼 동작하는 코드를 원하는 경우 기본 전략을 사용하세요.

지금까지 다양한 전략과 이를 인스턴스화하는 방법에 대해 배웠습니다. 다음 몇 가지 섹션은 훈련을 분산하는 데 사용할 수 있는 다양한 방법을 보여줍니다.

## Keras Model.fit과 함께 tf.distribute.Strategy 사용하기

`tf.distribute.Strategy`는 [Keras API 사양](https://keras.io)의 TensorFlow 구현인 `tf.keras`에 통합되었습니다. `tf.keras`는 모델을 빌드하고 훈련하기 위한 고수준 API입니다. `tf.keras` 백엔드에 통합하면 [Model.fit을 사용](/keras/customizing_what_happens_in_fit.ipynb)하여 Keras 훈련 프레임워크로 작성된 교육을 원활하게 분산할 수 있습니다.

코드에서 변경해야 할 사항은 다음과 같습니다.

1. 적절한 `tf.distribute.Strategy`의 인스턴스를 생성합니다.
2. Keras 모델, 옵티마이저 및 메트릭 생성을 `strategy.scope` 내로 이동합니다.

TensorFlow 분산 전략은 모든 Keras 모델 유형, 즉 [Sequential](/keras/sequential_model.ipynb), [Functional](/keras/functional.ipynb), [서브 클래스](/keras/custom_layers_and_models.ipynb)를 지원합니다.

다음은 하나의 `Dense` 레이어를 가진 매우 간단한 Keras 모델에 대해 이 작업을 수행하는 코드 조각입니다.

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()

with mirrored_strategy.scope():
  model = tf.keras.Sequential([tf.keras.layers.Dense(1, input_shape=(1,))])

model.compile(loss='mse', optimizer='sgd')

이 예제는 `MirroredStrategy`를 사용하기에 하나의 머신이 여러 GPU를 가진 경우에 실행할 수 있습니다. `strategy.scope()`는 훈련을 분산할 때 사용할 전략을 Keras에 표시합니다. 이 범위 내에서 모델/옵티마이저/메트릭을 생성하면 일반 변수 대신 분산 변수를 생성할 수 있습니다. 설정이 완료되면 평소처럼 모델을 맞출 수 있습니다. `MirroredStrategy`는 사용 가능한 GPU에서 모델의 훈련을 복제하고, 그래디언트를 집계하는 등의 작업을 수행합니다.

In [ ]:
dataset = tf.data.Dataset.from_tensors(([1.], [1.])).repeat(100).batch(10)
model.fit(dataset, epochs=2)
model.evaluate(dataset)

여기에서 `tf.data.Dataset`은 훈련 및 평가 입력을 제공합니다. NumPy 배열을 사용할 수도 있습니다.

In [ ]:
import numpy as np

inputs, targets = np.ones((100, 1)), np.ones((100, 1))
model.fit(inputs, targets, epochs=2, batch_size=10)

`Dataset` 또는 NumPy를 사용하는 경우 제공된 입력의 각 배치가 여러 복제본에 균등하게 분할됩니다. 예를 들어 2개의 GPU와 함께 `MirroredStrategy`를 사용하는 경우 크기가 10인 각 배치는 2개의 GPU로 분할되어 각 단계에서 5개의 입력 예제를 수신하게 됩니다. 그러면 GPU를 추가할 때마다 각 에포크가 더 빠르게 훈련됩니다. 일반적으로 가속기를 추가할 때 추가 계산 성능을 효과적으로 사용하기 위해 배치 크기를 늘리고 싶어할 것입니다. 또한 모델에 따라 학습률을 다시 조정할 필요가 있게 됩니다. 이러한 경우 `strategy.num_replicas_in_sync`를 사용하여 복제본 수를 가져올 수 있습니다.

In [ ]:
# Compute a global batch size using a number of replicas.
BATCH_SIZE_PER_REPLICA = 5
global_batch_size = (BATCH_SIZE_PER_REPLICA *
                     mirrored_strategy.num_replicas_in_sync)
dataset = tf.data.Dataset.from_tensors(([1.], [1.])).repeat(100)
dataset = dataset.batch(global_batch_size)

LEARNING_RATES_BY_BATCH_SIZE = {5: 0.1, 10: 0.15}
learning_rate = LEARNING_RATES_BY_BATCH_SIZE[global_batch_size]

### 현재 지원되는 것은 무엇입니까?

훈련 API | `MirroredStrategy` | `TPUStrategy` | `MultiWorkerMirroredStrategy` | `ParameterServerStrategy` | `CentralStorageStrategy`
--- | --- | --- | --- | --- | ---
Keras `Model.fit` | 지원 | 지원 | 지원 | 실험 기능으로 지원 | 실험 기능으로 지원

### 예제 및 튜토리얼

다음은 Keras `Model.fit`과의 엔드 투 엔드 통합을 보여주는 튜토리얼 및 예제 목록입니다.

1. [튜토리얼](../tutorials/distribute/keras.ipynb): {code 1}Model.fit{/code 1} 및 `MirroredStrategy`를 사용하는 훈련.
2. [튜토리얼](../tutorials/distribute/multi_worker_with_keras.ipynb): `Model.fit` 및 `MultiWorkerMirroredStrategy`를 사용하는 훈련.
3. [가이드](tpu.ipynb): `Model.fit` 및 `TPUStrategy` 사용 예제 포함.
4. [튜토리얼](../tutorials/distribute/parameter_server_training.ipynb): `Model.fit` 및 `ParameterServerStrategy`를 사용하는 매개변수 서버 훈련.
5. [튜토리얼](https://www.tensorflow.org/text/tutorials/bert_glue): `Model.fit` 및 `TPUStrategy`를 사용하여 다수의 GLUE 벤치마크 작업의 BERT를 미세 조정.
6. 다양한 전략을 사용하여 구현한 최신 모델 컬렉션이 포함된 TensorFlow Model Garden [리포지토리](https://github.com/tensorflow/models/tree/master/official).

## 사용자 정의 훈련 루프와 함께 tf.distribute.Strategy 사용하기

위에서 설명한 것처럼 Keras `Model.fit`과 함께 `tf.distribute.Strategy`를 사용하려면 몇 줄의 코드만 변경하면 됩니다. 조금만 더 노력하면 [사용자 정의 훈련 루프와 함께](/keras/writing_a_training_loop_from_scratch.ipynb) `tf.distribute.Strategy`를 사용할 수도 있습니다.

Estimator 또는 Keras에서 가능한 것보다 더 많은 유연성과 훈련 루프 제어가 필요한 경우 사용자 정의 훈련 루프를 작성할 수 있습니다. 예를 들어 GAN을 사용할 때 각 라운드마다 다른 수의 생성기 또는 판별기 단계를 수행할 수 있습니다. 마찬가지로 고수준 프레임워크는 강화 학습 훈련에 적합하지 않습니다.

`tf.distribute.Strategy` 클래스는 사용자 정의 훈련 루프를 지원하는 코어 메서드 세트를 제공합니다. 이를 사용하려면 초기에 코드를 약간 재구성해야 할 수 있지만 일단 재구성을 완료한면 전략 인스턴스를 변경하는 방식으로 GPU, TPU 및 여러 시스템 간에 전환할 수 있게 됩니다.

다음을 통해 이전과 동일한 Keras 모델을 사용하는 간단한 훈련 예제에 대한 이 사용 사례를 확인할 수 있습니다.


먼저 전략 범위 내에서 모델과 최적화 프로그램을 생성합니다. 이렇게 하면 모델과 옵티마이저로 생성한 모든 변수가 미러링된 변수가 됩니다.

In [ ]:
with mirrored_strategy.scope():
  model = tf.keras.Sequential([tf.keras.layers.Dense(1, input_shape=(1,))])
  optimizer = tf.keras.optimizers.SGD()

그런 다음 입력 데이터세트를 생성하고 `tf.distribute.Strategy.experimental_distribute_dataset`를 호출하여 전략에 따라 데이터세트를 분산합니다.

In [ ]:
dataset = tf.data.Dataset.from_tensors(([1.], [1.])).repeat(100).batch(
    global_batch_size)
dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)

그런 다음 훈련의 한 단계를 정의합니다. `tf.GradientTape`를 사용하여 그래디언트를 계산하고 옵티마이저를 사용하여 이러한 그래디언트를 적용하여 모델의 변수를 업데이트합니다. 이 훈련 단계를 배포하려면 위에서 업데이트한 변수를 `train_step` 함수에 넣고 이전에 생성한 `dist_dataset`로부터 가져온 데이터세트 입력과 함께 `tf.distribute.Strategy.run`로 전달합니다.

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(
  from_logits=True,
  reduction=tf.keras.losses.Reduction.NONE)

def compute_loss(labels, predictions):
  per_example_loss = loss_object(labels, predictions)
  return tf.nn.compute_average_loss(per_example_loss, global_batch_size=global_batch_size)

def train_step(inputs):
  features, labels = inputs

  with tf.GradientTape() as tape:
    predictions = model(features, training=True)
    loss = compute_loss(labels, predictions)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss

@tf.function
def distributed_train_step(dist_inputs):
  per_replica_losses = mirrored_strategy.run(train_step, args=(dist_inputs,))
  return mirrored_strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses,
                         axis=None)

위의 코드에서 주의해야 할 몇 가지 사항은 다음과 같습니다.

1. `tf.nn.compute_average_loss`를 사용하여 손실을 계산했습니다. `tf.nn.compute_average_loss`는 예제당 손실을 합산하고 합산 값을 `global_batch_size`로 나눕니다. 이 계산의 결과는 각 복제본으로 그래디언트를 계산한 후 이를 **합산**하는 방식으로 복제본들의 결과를 집계하기 때문에 중요합니다.
2. 또한 `tf.distribute.Strategy.reduce` API를 사용하여 `tf.distribute.Strategy.run`에서 반환한 결과를 집계했습니다. `tf.distribute.Strategy.run`은 전략의 각 로컬 복제본으로부터 입수한 결과를 반환하며 이 결과를 사용하는 방법에는 여러 가지가 있습니다. 사용자는 이를 `reduce`하여 집계된 값을 얻을 수 있습니다. `tf.distribute.Strategy.experimental_local_results`를 수행하여 결과에 포함된 값 목록을 로컬 복제본당 하나씩 가져올 수도 있습니다.
3. 분산 전략 범위 내에서 `apply_gradients`를 호출하면 동작이 수정됩니다. 특히, 동기식 훈련 동안 각 병렬 인스턴스에 그래디언트를 적용하기 전에 그래디언트의 전체 복제본 합계를 수행합니다.


마지막으로 훈련 단계를 정의한 후에 다음과 같이 `dist_dataset`를 반복하고 루프에서 훈련을 실행할 수 있습니다.

In [ ]:
for dist_inputs in dist_dataset:
  print(distributed_train_step(dist_inputs))

위의 예제에서는 `dist_dataset`를 반복하여 훈련에 입력을 제공했습니다. NumPy 입력을 지원하기 위해 `tf.distribute.Strategy.make_experimental_numpy_dataset`도 제공됩니다. `tf.distribute.Strategy.experimental_distribute_dataset`를 호출하기 전에 이 API를 사용하여 데이터세트를 생성할 수 있습니다.

데이터를 반복하는 또 다른 방법은 명시적으로 반복기를 사용하는 것입니다. 전체 데이터세트를 반복하는 것과는 대조적으로 제공된 단계 수만큼 실행하려는 경우 이 작업을 수행할 수 있습니다. 이제 위의 반복구문이 수정되어 먼저 반복기를 만들면 다음에는 명시적으로 `next`를 호출하여 입력 데이터를 가져옵니다.

In [ ]:
iterator = iter(dist_dataset)
for _ in range(10):
  print(distributed_train_step(next(iterator)))

여기서는 사용자 정의 훈련 루프를 분산하기 위해 `tf.distribute.Strategy` API를 사용하는 가장 간단한 경우를 다룹니다.

### 현재 지원되는 것은 무엇입니까?

훈련 API | `MirroredStrategy` | `TPUStrategy` | `MultiWorkerMirroredStrategy` | `ParameterServerStrategy` | `CentralStorageStrategy`
:-- | :-- | :-- | :-- | :-- | :--
사용자 정의 훈련 루프 | 지원 | 지원 | 지원 | 실험 기능으로 지원 | 실험 기능으로 지원

### 예제 및 튜토리얼

사용자 정의 훈련 루프와 함께 분산 전략을 사용하는 몇 가지 예제는 다음과 같습니다.

1. [튜토리얼](../tutorials/distribute/custom_training.ipynb): 사용자 정의 훈련 루프 및 `MirroredStrategy`를 사용하는 훈련.
2. [튜토리얼](../tutorials/distribute/multi_worker_with_ctl.ipynb): 사용자 정의 훈련 루프 및 `MultiWorkerMirroredStrategy`를 사용하는 훈련.
3. [가이드](tpu.ipynb): `TPUStrategy`를 사용하는 사용자 정의 훈련 루프 예제 포함.
4. [튜토리얼](../tutorials/distribute/parameter_server_training.ipynb): 사용자 정의 훈련 루프와 `ParameterServerStrategy`를 사용하는 매개변수 서버 훈련.
5. 다양한 전략을 사용하여 구현한 최신 모델 컬렉션이 포함된 TensorFlow Model Garden [리포지토리](https://github.com/tensorflow/models/tree/master/official).


## 기타 주제

이 섹션에서는 여러 사용 사례와 관련된 몇 가지 주제를 다룹니다.

<a name="TF_CONFIG"></a>

### TF_CONFIG 환경 변수 설정하기

다중 작업자 훈련의 경우 앞서 언급했듯이 클러스터에서 실행하는 각 바이너리에 대해 `'TF_CONFIG'` 환경 변수를 설정해야 합니다. `'TF_CONFIG'` 환경 변수는 클러스터를 구성하는 작업, 해당 주소 및 클러스터에서 각 작업의 역할을 지정하는 JSON 문자열입니다. [`tensorflow/ecosystem`](https://github.com/tensorflow/ecosystem) 리포지토리는 훈련 작업용 `'TF_CONFIG'`를 설정하는 Kubernetes 템플릿을 제공합니다.

`'TF_CONFIG'`에는 클러스터와 작업 등 두 가지 구성요소가 있습니다.

- 클러스터는 작업자와 같이 서로 다른 유형의 작업으로 구성된 사전인 훈련 클러스터에 대한 정보를 제공합니다. 다중 작업자 훈련에는 일반 작업자가 수행하는 작업 외에 TensorBoard에 대한 체크포인트 저장 및 요약 파일 작성과 같은 약간의 더 많은 책임을 수행하는 작업자가 일반적으로 하나 있습니다. 이러한 작업자를 "수석" 작업자라고 하며 인덱스가 `0`인 작업자를 수석 작업자로 지정하는 것이 관례입니다(실제로 이것은 `tf.distribute.Strategy`이 구현되는 방식임).
- 반면에 작업은 현재 작업에 대한 정보를 제공합니다. 첫 번째 구성 요소 클러스터는 모든 작업자에 대해 동일하지만 두 번째 구성 요소 작업은 작업자마다 다르며 해당 작업자의 유형과 인덱스를 지정합니다.

`'TF_CONFIG'`의 예제는 다음과 같습니다.

```python
os.environ["TF_CONFIG"] = json.dumps({
    "cluster": {
        "worker": ["host1:port", "host2:port", "host3:port"],
        "ps": ["host4:port", "host5:port"]
    },
   "task": {"type": "worker", "index": 1}
})
```


이 `'TF_CONFIG'`는 해당하는 호스트 및 포트와 함께 `"cluster"`에 3개의 작업자와 2개의 `"ps"` 작업이 있음을 지정합니다. `"task"` 부분은 `"cluster"`, 즉 작업자 `1`(두 번째 작업자)에 현재 작업의 역할을 지정합니다. 클러스터의 유효한 역할은 `"chief"`, `"worker"`, `"ps"` 및 `"evaluator"`입니다. `tf.distribute.experimental.ParameterServerStrategy`를 사용하는 경우 외에는 `"ps"` 작업이 없어야 합니다.

## 다음 단계

`tf.distribute.Strategy`는 현재 개발 중입니다. 사용해 보고 [GitHub 문제](https://github.com/tensorflow/tensorflow/issues/new)를 사용하여 피드백을 제공해주세요.